# Classification of messages as spam or not spam using Naive Bayes algorithm 

In [1]:
import pandas as pd

In [2]:
# Import Dataset
df = pd.read_table('SMS', sep='\t', header=None,names=['label', 'sms_message'])
df

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
# map the 'ham' value to 0 and the 'spam' value to 1.
df['categories'] = df.label.map({'ham':0,'spam':1})

In [4]:
# Randomize the dataset
df = df.sample(frac=1, random_state=1)
df

,label,sms_message,categories
1078,ham,"Yep, by the pretty sculpture",0
4028,ham,"Yes, princess. Are you going to make me moan?",0
958,ham,Welp apparently he retired,0
4642,ham,Havent.,0
4674,ham,I forgot 2 ask ü all smth.. There's a card on ...,0
...,...,...,...
905,ham,"We're all getting worried over here, derek and...",0
5192,ham,Oh oh... Den muz change plan liao... Go back h...,0
3980,ham,CERI U REBEL! SWEET DREAMZ ME LITTLE BUDDY!! C...,0
235,spam,Text & meet someone sexy today. U can find a d...,1


In [6]:
# Split into training and test sets
training_test_index = round(len(df) * 0.8)

training = df[:training_test_index].reset_index(drop=True)
test = df[training_test_index:].reset_index(drop=True)

print(training.shape)
print(test.shape)

(4458, 3)
(1114, 3)


In [8]:
#  data cleaning 
training['sms_message'] = training['sms_message'].str.replace('\W', ' ') # Removes punctuation
training['sms_message'] = training['sms_message'].str.lower() ### making all the words lowercase

test['sms_message'] = test['sms_message'].str.replace('\W', ' ') # Removes punctuation
test['sms_message'] = test['sms_message'].str.lower() ### making all the words lowercase
training

,label,sms_message,categories
0,ham,yep by the pretty sculpture,0
1,ham,yes princess are you going to make me moan,0
2,ham,welp apparently he retired,0
3,ham,havent,0
4,ham,i forgot 2 ask ü all smth there s a card on ...,0
...,...,...,...
4453,ham,sorry i ll call later in meeting any thing re...,0
4454,ham,babe i fucking love you too you know fuck...,0
4455,spam,u ve been selected to stay in 1 of 250 top bri...,1
4456,ham,hello my boytoy geeee i miss you already a...,0


In [9]:
### creating vocabulary from training data
training['sms_message'] = training['sms_message'].str.split()
vocabulary = []
for sms in training['sms_message']:
   for word in sms:
      vocabulary.append(word)
vocabulary = list(set(vocabulary))  ### only count the number of unique words
print(len(vocabulary))

7783


In [10]:
word_counts_per_sms = {unique_word: [0] * len(training['sms_message']) for unique_word in vocabulary}

for index, sms in enumerate(training['sms_message']):
   for word in sms:
      word_counts_per_sms[word][index] += 1
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts

,tcs,connect,defo,supply,08715203652,morning,summon,123,prakesh,terms,...,brilliantly,08448350055,manda,ibn,apt,give,tok,snowball,rpl,clocks
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
training_new = pd.concat([training, word_counts], axis=1)
training_new.head()

,label,sms_message,categories,tcs,connect,defo,supply,08715203652,morning,summon,...,brilliantly,08448350055,manda,ibn,apt,give,tok,snowball,rpl,clocks
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Laplace smoothing
alpha = 1

# You will start from here.

In [61]:
# Hints:
# Step 1: caculate P(Spam) and P(Ham)
# Step 2: count N_Spam, N_Ham, N_Vocabulary
# Step 3: count the number of times the word w occurs in spam/ham message: N_w_spam
# Step 4: p(w|spam)=(N_w_spam+alpha)/(N_Spam+alpha*N_Vocabulary)
#         p(w|Ham)=(N_w_ham+alpha)/(N_Ham+alpha*N_Vocabulary)

# Calculate accuracy, precision, recall and F1_score. 

In [ ]:
# Model Evaluation 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy score: {}'.format(accuracy_score(test['categories'], test['predicted'])))
print('Precision score: {}'.format(precision_score(test['categories'], test['predicted'])))
print('Recall score: {}'.format(recall_score(test['categories'], test['predicted'])))
print('F1 score: {}'.format(f1_score(test['categories'], test['predicted'])))